In [1]:
# import matplotlib.pyplot as plt
# import matplotlib
# from matplotlib.pyplot import figure
import numpy as np
import pandas as pd
# import dataframe_image as dfi
# from scipy.stats import linregress
import pickle

# plt.rcParams.update({'font.size': 20})

# matplotlib.rc('font', family='sans-serif') 
# matplotlib.rc('font', serif='Helvetica') 
# matplotlib.rc('text', usetex='false') 

First define data class and its functions.

In [2]:
class tc():
    def __init__(self, filename, vg_min, vg_max, step, shift, vds=0.1, linear_fit_steps=[10]):
        # linear_fit_steps: list
        # shift: list
        # self.steps = linear_fit_steps
        # self.shift = shift
        self.load(filename, vg_min, vg_max, step, shift, vds, linear_fit_steps, type)
        self.to_pandas(filename)
        
        
    def load(self, filename,vg_min, vg_max, step, shift, vds, linear_fit_steps, type):
        raw_data = pd.read_csv(filename + ".csv", header=None).to_numpy().T
        raw_data[1] = raw_data[1]/vds
        data_up = raw_data[:,:step]
        data_dn = np.flip(raw_data[:,step:], axis=1)
        self.up = tc_single(data_up, vg_min, vg_max, step, shift, linear_fit_steps)
        self.dn = tc_single(data_dn, vg_min, vg_max, step, shift, linear_fit_steps)

    def to_pandas(self, filename):
        df_up_n = self.up.n.params
        df_up_p = self.up.p.params
        df_dn_n = self.dn.n.params
        df_dn_p = self.dn.p.params
        self.params = pd.concat([df_dn_n,df_dn_p,df_up_n,df_up_p], keys=['dn_N', 'dn_P',"up_N",'up_P'])
        self.params.to_html(filename + '_params.html')
        
class tc_single():
    def __init__(self, data, vg_min, vg_max, step, shift, linear_fit_steps):
        p_data = np.zeros(data.shape)
        p_data[0] = np.flip(-data[0])
        p_data[1] = np.flip(data[1])
        self.n = tc_single_type(data, vg_min, vg_max, step, shift, linear_fit_steps)
        self.p = tc_single_type(p_data, -vg_max, -vg_min, step, shift, linear_fit_steps)
        
class tc_single_type():
    def __init__(self, data, vg_min, vg_max, step, shift, linear_fit_steps):
        self.vg_min = vg_min
        self.vg_max= vg_max
        self.step = step
        self.shift = shift
        self.fit_steps = linear_fit_steps 
        self.data = data
        
        # self.data is a nparray, of shape (2, step)
        self.ids = np.zeros(len(linear_fit_steps))
        self.mu_max = np.zeros(len(linear_fit_steps))
        self.vt = np.zeros(len(linear_fit_steps))
        self.ss = np.zeros(len(linear_fit_steps))
        self.vt_fit_data = []
        self.fit()
        self.mu = np.zeros((len(linear_fit_steps), len(shift)))
        self.shift_fit_data = [[] for _ in range(len(self.fit_steps))]
        self.shift_fit()
        
        self.to_pandas()
        
    def lin_reg(self, data):
        slope, intercept =  np.polyfit(data[0],data[1],1)
        mu = np.abs(slope/1.2E-8)
        vt = -intercept/slope
        ids = np.mean(data[1])
        ss, _ =  np.polyfit(np.log(data[1]),data[0],1)
        return mu, vt, ids, ss, slope
    
    def fit(self):
        for i,step in enumerate(self.fit_steps):
            lin_data = self.data[:,-step:]
            self.mu_max[i], self.vt[i], self.ids[i], self.ss[i], slope = self.lin_reg(lin_data)
            vt_fit_data = np.zeros(self.data.shape)
            vt_fit_data[0] = self.data[0]
            vt_fit_data[1] = slope * (self.data[0] - self.vt[i])
            mask = (vt_fit_data[1] >=0)
            self.vt_fit_data.append(vt_fit_data[:,mask])
    
    def shift_fit(self):
        for i,step in enumerate(self.fit_steps):
            for j,sf in enumerate(self.shift):
                if self.vt[i] + sf + step < self.vg_max:
                    mask = (self.data[0] >= self.vt[i]+ sf) & (self.data[0] <= self.vt[i] + sf + step)
                    lin_data = self.data[:,mask]
                    self.mu[i,j], _, _, _, _ = self.lin_reg(lin_data)
                    self.shift_fit_data[i].append(lin_data)
                    # print("Normal")
                    # print(lin_data.shape)
                elif self.vt[i]+ sf < self.vg_max:
                    mask = (self.data[0] >= self.vt[i]+ sf) & (self.data[0] <= self.vg_max)
                    lin_data = self.data[:,mask]
                    self.mu[i,j], _, _, _, _ = self.lin_reg(lin_data)
                    self.shift_fit_data[i].append(lin_data)
                    # print("Not normal")
                    # print(lin_data.shape)
                else:
                    self.mu[i,j] = np.nan
                    
    def to_pandas(self):
        df = pd.DataFrame(columns=["Step_size_"+str(step) for step in self.fit_steps])
        df.loc['Vt'] = self.vt
        df.loc['mu_max'] = self.mu_max 
        for i,mu in enumerate(self.mu.T):
            df.loc['mu_shift_'+str(self.shift[i]) + '_steps'] = mu
        self.params = df

def save_write_result(result:tc, name_pickle:str, name_txt:str):
    file = open(name_pickle,"wb")
    pickle.dump(result, file)
    file.close()
    


In [95]:
test = tc(filename='test', 
   vg_max=50,
   vg_min=-50,
   shift=[15,20,30],
   linear_fit_steps=[5,10,12,15,17],
   step=101)

In [96]:
# file = open("test_pickle","wb")
# pickle.dump(test, file)
# file.close()

# file = open("test_pickle", "rb")
# test_read = pickle.load(file)
# file.close()

In [4]:
file = open("test_pickle", "rb")
test_read = pickle.load(file)
file.close()